# Tarea 5. Modelos de Lenguaje Nauronales

Guillermo Segura Gómez

## Ejercicio 1

**1. Con base en la implementación mostrada en las prácticas del NLM, construya un modelo de lenguaje neuronal a nivel de carácter. Tomé en cuenta secuencias de tamaño 6 o más para el modelo, es decir hasta 5 caracteres o más en el contexto. Ponga al modelo a generar texto 3 veces, con un máximo de 300 caracteres. Escriba 5 ejemplos de oraciones y mídales el likelihood. Escriba un ejemplo de estructura morfológica (permutaciones con caracteres) similar al de estructura sintáctica del profesor con 5 o más caracteres de su gusto (e.g., "ando "). Calcule la perplejidad del modelo sobre los datos val.**

### Modelos de lenguaje neuronales

Los Modelos de Lenguaje Neuronal (NLM) son fundamentales en el campo del Procesamiento del Lenguaje Natural para generar texto, completar frases, y más (mucho más). Mientras que los modelos tradicionales a nivel de palabra capturan la estructura y semántica del lenguaje a partir de las palabras, los modelos a nivel de carácter ofrecen una mirada más profunda a la formación de palabras y estructuras sintácticas. Al enfocarse en caracteres, estos modelos pueden generalizar mejor en idiomas con una rica morfología o en casos donde la ortografía y la formación de palabras juegan un papel crucial. 

Construiremos un NLM a nivel de carácter, lo utilizaremos para generar texto, evaluaremos ejemplos de oraciones mediante el cálculo de su log-verosimilitud, exploraremos estructuras morfológicas mediante permutaciones de caracteres, y mediremos la perplejidad del modelo en un conjunto de datos de validación. 

Comenzamos siguiendo el procedimiento descrito en la práctica 5 de NLM.

In [1]:
# General libraries
import os
import time
import shutil
import random
from typing import Tuple
from argparse import Namespace
from itertools import chain

import matplotlib.pyplot as plt

#Preprocessing
import nltk
from nltk.corpus import stopwords
from nltk import ngrams
from nltk.tokenize import TweetTokenizer
from nltk.util import ngrams
from nltk import FreqDist
import pandas as pd
import numpy as np

# PyTorch
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
import torch.nn.functional as F

#scikit-learn
from sklearn.metrics import accuracy_score

Definimos los parámetros generadores.

In [2]:
seed = 1111
random.seed(seed) #python seed
np.random.seed(seed) #numpy seed
torch.manual_seed(seed) #torch seed
torch.backends.cudnn.benchmark = False 

Inicializamos los datos en un data frame de pandas.

In [3]:
path_text = "/Users/guillermo_sego/Desktop/Segundo Semestre/PLN/Data/MexData/mex20_train.txt"
path_text_val = "/Users/guillermo_sego/Desktop/Segundo Semestre/PLN/Data/MexData/mex20_val.txt"

X_train = pd.read_csv(path_text, sep = '\r\n', engine = 'python', header = None).loc[:, 0].values.tolist()
X_val = pd.read_csv(path_text_val, sep = '\r\n', engine = 'python', header = None).loc[:, 0].values.tolist()

Definimos la clase `CaracterData` está diseñada para procesar datos textuales para tareas de modelado de lenguaje o similares, utilizando un enfoque basado en caracteres. El propósito de esta clase incluye varias funciones importantes para el preprocesamiento de datos textuales y su preparación para algoritmos de aprendizaje automático. 

In [4]:
class CaracterData():
    def __init__(self, N:int, vocab_max: int = None):
        self.N = N  # Tamaño de la secuencia de caracteres, incluyendo el carácter objetivo
        self.vocab_max = vocab_max  # Limitar el tamaño del vocabulario, si es necesario
        self.UNK = '<unk>'  # Representación de caracteres desconocidos
        self.vocab = set()

    def get_vocab(self, corpus: list) -> set:
        # Construye un conjunto de caracteres únicos a partir del corpus
        unique_chars = set(chain(*corpus))
        return unique_chars

    def fit(self, corpus: list) -> None:
        # Construye el vocabulario a partir del corpus
        self.vocab = self.get_vocab(corpus)
        
        # Crea mapeos de caracteres a índices y viceversa
        self.char2id = {char: idx for idx, char in enumerate(self.vocab)}
        self.id2char = {idx: char for char, idx in self.char2id.items()}

        # Incluir el token UNK en el vocabulario y los mapeos
        self.char2id[self.UNK] = len(self.char2id)
        self.id2char[len(self.id2char)] = self.UNK
    
    def get_vocab_size(self) -> int:
        # Retorna el tamaño del conjunto vocab
        return len(self.vocab)

    def transform(self, corpus: list) -> Tuple[np.ndarray, np.ndarray]:
        X_ngrams, y = [], []

        for doc in corpus:
            doc_ngrams = list(ngrams(doc, self.N))
            for ngram in doc_ngrams:
                # Usa el índice para UNK si el carácter no está en char2id
                X_ngrams.append([self.char2id.get(char, self.char2id[self.UNK]) for char in ngram[:-1]])
                y.append(self.char2id.get(ngram[-1], self.char2id[self.UNK]))

        return np.array(X_ngrams), np.array(y)

    def get_ngram_doc(self, doc: str) -> list:
        # Genera n-gramas del documento
        return list(ngrams(doc, self.N))

    def replace_unk(self, doc_tokens: list) -> list:
        # Reemplaza caracteres desconocidos por un índice especial, si es necesario
        return [token if token in self.char2id else '<unk>' for token in doc_tokens]

In [5]:
args = Namespace()
args.N = 6  # Tamaño de secuencia deseado

# Creamos un objeto con la clase adaptada para caracteres
caracter_data = CaracterData(args.N)

# Ajustamos el modelo a nuestro conjunto de datos de entrenamiento para construir el vocabulario de caracteres
# Nota: No es necensario pasar un tokenizador ya que estamos trabajando a nivel de carácter
caracter_data.fit(X_train)

Ahora utilizamos los métodos de la clase que creamos. El método `transform` transforma los datos textuales proporcionados en un formato que es adecuado para el entrenamiento o la evaluación en modelos de aprendizaje automático.

In [6]:
X_caracter_train, y_caracter_train = caracter_data.transform(X_train)
X_caracter_val, y_caracter_val = caracter_data.transform(X_val)

In [7]:
print(f'Vocab size: {caracter_data.get_vocab_size()}')

Vocab size: 425


Ahora necesitamos pasar estos datos en tensores de PyTorch y, organizarlos en un DataLoader de PyTorch para facilitar el entrenamiento y la evaluación de modelos de aprendizaje profundo. PyTorch utiliza tensores, que son una generalización de matrices y vectores, como su estructura de datos principal para realizar operaciones de aprendizaje automático, especialmente en redes neuronales.

Para el entrenamiento, es útil organizar los datos en lotes (batches), y PyTorch ofrece una herramienta llamada **DataLoader** para esto. Un DataLoader puede cargar los datos en lotes de un tamaño especificado y puede mezclar los datos para reducir el riesgo de sobreajuste. Para usar un DataLoader, primero debes organizar tus tensores en un Dataset, que es otra abstracción de PyTorch que facilita trabajar con conjuntos de datos.

In [8]:
# Set batch size in args
args.batch_size = 64
# Num workers
args.num_workers = 2

# Convertimos los datos a tensores de pytorch
train_dataset = TensorDataset(torch.tensor(X_caracter_train, dtype = torch.int64),
                              torch.tensor(y_caracter_train, dtype = torch.int64))

train_loader = DataLoader(train_dataset,
                          batch_size = args.batch_size,
                          num_workers=args.num_workers,
                          shuffle = True)

val_dataset = TensorDataset(torch.tensor(X_caracter_val, dtype = torch.int64),
                            torch.tensor(y_caracter_val, dtype = torch.int64))

val_loader = DataLoader(train_dataset,
                          batch_size = args.batch_size,
                          num_workers=args.num_workers,
                          shuffle = False)

In [9]:
batch = next(iter(train_loader))
print(f'X shape: {batch[0].shape}')
print(f'y shape: {batch[1].shape}')

X shape: torch.Size([64, 5])
y shape: torch.Size([64])


Parámetros que definen aspectos estructurales y de regularización de un modelo de red neuronal, que influyen en cómo el modelo aprenderá de los datos y generalizará a nuevos ejemplos no vistos. Utilizamos los parámetros del modelo propuesto por Bengio.

In [10]:
# Vocab size
args.vocab_size = caracter_data.get_vocab_size()

# Dimension of word embeddings
args.d = 50

# Dimension for hidden layer
args.d_h = 100

# Dropout
args.dropout = 0.1

Definimos la clase `NeuralLM`, que representa un Modelo de Lenguaje Neuronal, utilizando PyTorch. Esta clase hereda de `nn.Module`, que es la clase base para todos los módulos de red neuronal en PyTorch, proporcionando funcionalidades útiles como el seguimiento de parámetros, la GPU/CPU transferencia, etc.

In [11]:
class NeuralLM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, window_size, dropout):
        super(NeuralLM, self).__init__()
        
        # Tamaño de la ventana de entrada para el modelo, correspondiente al contexto de caracteres
        self.window_size = window_size - 1  # Menos 1 porque el último carácter es el objetivo
        
        # Dimensión de los embeddings de caracteres
        self.embedding_dim = embedding_dim
        
        # Capa de embedding que convierte índices de caracteres en vectores densos
        self.emb = nn.Embedding(vocab_size, self.embedding_dim)
        
        # Primera capa lineal que transforma la entrada de embeddings aplanada a una representación intermedia
        self.fc1 = nn.Linear(self.embedding_dim * self.window_size, hidden_dim)
        
        # Capa de Dropout para reducir el sobreajuste
        self.drop1 = nn.Dropout(p=dropout)
        
        # Segunda capa lineal que transforma la salida de la capa oculta en logits para cada carácter en el vocabulario
        self.fc2 = nn.Linear(hidden_dim, vocab_size, bias=False)
        
    def forward(self, x):
        # Transforma los índices de caracteres en embeddings
        x = self.emb(x)
        
        # Aplana los embeddings en un vector único para cada muestra
        # print(self.window_size * self.embedding_dim)
        x = x.view(-1, self.window_size * self.embedding_dim)
        
        # Aplica la primera capa lineal y una función de activación ReLU
        h = F.relu(self.fc1(x))
        
        # Aplica Dropout a la representación de la capa oculta
        h = self.drop1(h)
        
        # Genera los logits para cada carácter en el vocabulario
        return self.fc2(h)

Ahora necesitamos mas funciones para poder hacer la evaluación del modelo.

In [12]:
# Esta función toma los logits (es decir, las salidas no normalizadas de un modelo) 
# y devuelve las predicciones de clase como índices.
def get_preds(raw_logits):
    # Calcula las probabilidades aplicando la función softmax a los logits.
    # La operación detach() se usa para evitar que se calculen gradientes para estas operaciones,
    # ya que solo se necesitan las probabilidades para hacer predicciones.
    probs = F.softmax(raw_logits.detach(), dim=1)
    
    # Encuentra el índice de la mayor probabilidad en cada fila (es decir, para cada ejemplo en el lote),
    # que corresponde a la clase predicha. Luego, convierte el tensor a un array de NumPy.
    y_pred = torch.argmax(probs, dim=1).cpu().numpy()
    
    return y_pred


# Evalúa el modelo en un conjunto de datos proporcionado y devuelve la precisión del modelo.
def model_eval(data, model, gpu=False):
    # Desactiva el cálculo de gradientes para acelerar las cosas y reducir el uso de memoria
    # ya que no se necesita para la evaluación.
    with torch.no_grad():
        preds, tgts = [], []  # Listas para almacenar predicciones y etiquetas verdaderas
        
        # Itera sobre los lotes de datos en el DataLoader
        for window_words, labels in data:
            # Si se utiliza GPU, mueve los datos al dispositivo adecuado
            if gpu:
                window_words = window_words.cuda()
                
            # Obtiene los logits del modelo para el lote actual
            outputs = model(window_words)
            
            # Obtiene las predicciones de clase para el lote actual utilizando la función get_preds
            y_pred = get_preds(outputs)
            
            # Extrae las etiquetas verdaderas del lote actual y las convierte a un array de NumPy
            tgt = labels.numpy()
            
            # Almacena las predicciones y las etiquetas verdaderas
            tgts.append(tgt)
            preds.append(y_pred)
        
        # Aplana las listas de listas para obtener una única lista de etiquetas y predicciones
        tgts = [e for l in tgts for e in l]
        preds = [e for l in preds for e in l]
        
        # Calcula y devuelve la precisión del modelo comparando las predicciones con las etiquetas verdaderas
        return accuracy_score(tgts, preds)
    

# Guarda el estado actual del modelo y, si es el mejor modelo hasta el momento según 
# algún criterio, guarda una copia separada.
def save_checkpoint(state, is_best, checkpoint_path, filename="checkpoint.pt"):
    # Construye la ruta completa del archivo donde se guardará el estado del modelo
    filename = os.path.join(checkpoint_path, filename)
    
    # Guarda el estado del modelo en la ruta especificada
    torch.save(state, filename)
    
    # Si el modelo actual es el "mejor" según algún criterio, guarda una copia separada
    if is_best:
        # Copia el archivo del checkpoint al archivo del "mejor modelo"
        shutil.copyfile(filename, os.path.join(checkpoint_path, "model_best.pt"))



In [13]:
# Model hyperparameters
args. vocab_size = caracter_data.get_vocab_size()
args.d = 100
args.d_h = 200
args.dropout = 0.1

# Training hyperparameters
args.lr = 2.3e-1
args.num_epochs = 20
args.patience = 20

# Scheduler hyperparameters
args.lr_patience = 10
args.lr_factor = 0.5

# Saving directoty
args.savedir = 'model'
os.makedirs(args.savedir, exist_ok=True)

# Create model
model = NeuralLM(
    vocab_size=args.vocab_size,
    embedding_dim=args.d,
    hidden_dim=args.d_h,
    window_size=args.N,
    dropout=args.dropout
)

# Send to GPU
args.use_gpu = torch.cuda.is_available()
if args.use_gpu:
    model.cuda()
    
# Loss, optimizer an scheduler
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr= args.lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, 'min',
                patience=args.lr_patience,
                verbose=True,
                factor=args.lr_factor
                )

/Users/guillermo_sego/anaconda3/envs/PLN/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Ahora realizamos el entrenamiento y validación del modelo de aprendizaje profundo utilizando PyTorch, incluyendo características como el early stopping y el ajuste dinámico de hiperparámetros (scheduler).

In [14]:
start_time = time.time()
best_metric = 0
metric_history = []
train_metric_history = []

for epoch in range(args.num_epochs):
    epoch_stats_time = time.time()
    loss_epoch = []
    training_metric = []
    model.train()
    
    for window_words, labels in train_loader:
        if args.use_gpu:
            window_words = window_words.cuda()
            labels = labels.cuda()
            
        # Forward pass
        outputs = model(window_words)
        loss = criterion(outputs, labels)
        loss_epoch.append(loss.item())
        
        # Get training metrics
        y_pred = get_preds(outputs)
        tgt = labels.cpu().numpy()
        training_metric.append(accuracy_score(tgt, y_pred))
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    # Get metric in training dataset
    mean_epoch_metric = np.mean(training_metric)
    train_metric_history.append(mean_epoch_metric)
    
    # Get metric in validation dataset
    model.eval()
    tuning_metric = model_eval(val_loader, model, gpu=args.use_gpu)
    metric_history.append(mean_epoch_metric)
    
    # Update scheduler
    scheduler.step(tuning_metric)
    
    # Check for metric improvement
    is_improvement = tuning_metric > best_metric
    if is_improvement:
        best_metric = tuning_metric
        n_no_improve = 0
    else:
        n_no_improve = 1
        
    save_checkpoint(
        {
            "epoch": epoch + 1,
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict(),
            "scheduler": scheduler.state_dict(),
            "best_metric": best_metric,
        },
        is_improvement,
        args.savedir,
    )
    
    # Early stopping
    if n_no_improve >= args.patience:
        print("No improvement. Breaking out of loop.")
        
    print("Train acc: {}".format(mean_epoch_metric))
    print("Epoch [{}/{}], Loss: {:.4f} - Val accuracy: {:.4f} - Epoch time: {:.2f}"
         .format(epoch+1, args.num_epochs, np.mean(loss_epoch), tuning_metric, (time.time())))
    

print("--- %s seconds ---" % (time.time() - start_time))

Train acc: 0.40929519959521143
Epoch [1/20], Loss: 2.1188 - Val accuracy: 0.4563 - Epoch time: 1711170826.36
Train acc: 0.45270056412023085
Epoch [2/20], Loss: 1.9259 - Val accuracy: 0.4782 - Epoch time: 1711170844.05
Train acc: 0.46749324455257946
Epoch [3/20], Loss: 1.8610 - Val accuracy: 0.4906 - Epoch time: 1711170862.24
Train acc: 0.4761725142106623
Epoch [4/20], Loss: 1.8221 - Val accuracy: 0.4917 - Epoch time: 1711170880.25
Train acc: 0.4824375861252261
Epoch [5/20], Loss: 1.7942 - Val accuracy: 0.4979 - Epoch time: 1711170898.38
Train acc: 0.487113378477306
Epoch [6/20], Loss: 1.7733 - Val accuracy: 0.5036 - Epoch time: 1711170916.69
Train acc: 0.49012574283007493
Epoch [7/20], Loss: 1.7568 - Val accuracy: 0.5100 - Epoch time: 1711170934.65
Train acc: 0.49334198174145205
Epoch [8/20], Loss: 1.7444 - Val accuracy: 0.5134 - Epoch time: 1711170952.74
Train acc: 0.495884425329429
Epoch [9/20], Loss: 1.7321 - Val accuracy: 0.5144 - Epoch time: 1711170971.28
Train acc: 0.498376404917

### Evaluación del modelo

Utiizamos el modelo de lenguaje neuronal entrenado para generar texto de manera autónoma. Las siguientes funciones son adaptaciones para trabajar con caracteres de las funciones revisadas en la práctica 5. 

In [15]:
def parse_text(text, char2id):
    # Convierte el texto en caracteres y usa '<unk>' para caracteres desconocidos.
    all_chars = [char if char in char2id else '<unk>' for char in text]
    
    # Convierte los caracteres a sus índices numéricos correspondientes según el mapeo char2id.
    char_ids = [char2id[char] for char in all_chars]
    # print(len(char_ids))
    return all_chars, char_ids

def sample_next_word(logits, temperature=1.0):
    # Convierte los logits a un array de numpy y ajusta la "temperatura" de la predicción.
    logits = np.asarray(logits).astype("float64")
    preds = logits / temperature
    
    # Convierte los logits ajustados a probabilidades usando softmax.
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    
    # Muestrea un índice de palabra de la distribución de probabilidades.
    probas = np.random.multinomial(1, preds)
    return np.argmax(probas)

def predict_next_token(model, token_ids):
    # Convierte la lista de índices de tokens a un tensor de PyTorch y agrega una dimensión de lote.
    word_ids_tensor = torch.LongTensor(token_ids).unsqueeze(0)
    
    # Obtiene los logits de la predicción del modelo para la secuencia de tokens y los convierte a numpy.
    y_raw_pred = model(word_ids_tensor).squeeze(0).detach().numpy()
    
    # Muestra el índice de la siguiente palabra de la distribución de logits.
    y_pred = sample_next_word(y_raw_pred, 1.0)
    return y_pred

def generate_sentence(model, initial_text, char2id, nMax = 300):
    # Obtiene tokens y sus índices del texto inicial.
    all_tokens, window_word_ids = parse_text(initial_text, char2id)
    
    # Genera hasta 300 palabras adicionales.
    for i in range(nMax):
        # Predice el índice de la siguiente palabra utilizando el modelo.
        y_pred = predict_next_token(model, window_word_ids)

        next_word = caracter_data.id2char[y_pred]  # Convierte el índice de palabra predicho a texto.
        all_tokens.append(next_word)  # Añade la palabra predicha a la lista de tokens.
        
        # Si se genera el token de fin de secuencia, detiene la generación.
        if next_word == "</s>":
            break
        else:
            # Actualiza la ventana de palabras para la siguiente predicción.
            window_word_ids.pop(0)  # Elimina la primera palabra.
            window_word_ids.append(y_pred)  # Añade la nueva palabra al final.
    
    # Une los tokens generados en una cadena de texto y la devuelve.
    return "".join(all_tokens)


Generamos las cadenas de texto

In [16]:
# Model with learned embeddings from scratch
best_model = NeuralLM(
    vocab_size=args.vocab_size,
    embedding_dim=args.d,
    hidden_dim=args.d_h,
    window_size=args.N,
    dropout=args.dropout
)

best_model.load_state_dict(torch.load("model/model_best.pt")["state_dict"])
best_model

NeuralLM(
  (emb): Embedding(425, 100)
  (fc1): Linear(in_features=500, out_features=200, bias=True)
  (drop1): Dropout(p=0.1, inplace=False)
  (fc2): Linear(in_features=200, out_features=425, bias=False)
)

In [17]:
initial_text = "Cinco"
generated_text = generate_sentence(
    model=best_model,
    initial_text=initial_text,
    char2id=caracter_data.char2id,
    nMax=300
)

print("Texto generado:", generated_text)

Texto generado: Cincontrica que “hale el #Bese enamente te plrapita no 10 por a la esas rece novia en un chuna pendeja para el mayar o albe... Grana chingo lo idgo la verga dequerose q adiendo prieron, se va a estaban cosasQueMéQué día que sean la el ninero camociando la pello, quieren en las cosas.😑. A no S pendejos ne


In [18]:
initial_text = "Reino"
generated_text = generate_sentence(
    model=best_model,
    initial_text=initial_text,
    char2id=caracter_data.char2id,
    nMax=300
)

print("Texto generado:", generated_text)

Texto generado: Reino goconocible raja de gazados de demato. Y sus putas. .0 vídoma. (lt) y que tu caniendo recuperro como loca de que decen del de mierda, luchona noches nocia! Que hacen de Fax La creer. Bueno fuertro eran a noviantes draga!.!2 de ser con que tabas son Jaje Plaeta. Phone, sus vaba no lleves y soy de mi


In [19]:
initial_text = "Habia"
generated_text = generate_sentence(
    model=best_model,
    initial_text=initial_text,
    char2id=caracter_data.char2id,
    nMax=300
)

print("Texto generado:", generated_text)

Texto generado: Habiablo soy hace me diady... el podia, cosasse #Hombre gienda y no hago la lestodichamenlasea. No mame, oereeción en lo puto en Edf alcumudiado el Rolfos y despuestas nadie no hacen a ventar en bien a REIGOARAWO @USUARIO @USUARIO , el oficiendo en la solo que me vale, descumplas? Ingue eregó ellas su ed


La función `log_likelihood` calcula la log-verosimilitud de una secuencia de texto dada bajo un modelo de lenguaje entrenado. La log-verosimilitud es una medida de cuán probable es que el modelo haya generado una secuencia de texto dada.

In [20]:
def log_likelihood(model, text, caracter_model):
    # Transforma el texto
    X, y = caracter_model.transform([text])
    
    # Convierte X en un tensor de PyTorch y agrega una dimensión de lote, preparándolo para el modelo.
    X = torch.LongTensor(X).unsqueeze(0)
    
    # Obtiene los logits del modelo para el texto transformado y luego los desconecta del grafo de cómputo.
    logits = model(X).detach()
    
    # Aplica softmax a los logits para obtener una distribución de probabilidades sobre el vocabulario.
    probs = F.softmax(logits, dim=1).numpy()
    
    # Calcula la log-verosimilitud sumando los logaritmos de las probabilidades de las palabras reales (y)
    # según lo predicho por el modelo.
    return np.sum([np.log(probs[i][w]) for i, w in enumerate(y)])


In [21]:
print("log likelihood: ", log_likelihood(best_model, "Estamos en la clase de procesamiento de lenguaje", caracter_data))

log likelihood:  -69.76852


In [22]:
print("log likelihood: ", log_likelihood(best_model, "Había una vez en un lugar muy lejano", caracter_data))

log likelihood:  -51.269493


In [23]:
print("log likelihood: ", log_likelihood(best_model, "Eres un tonto viejo loco", caracter_data))

log likelihood:  -34.32611


In [24]:
print("log likelihood: ", log_likelihood(best_model, "Eres muy buena persona mi estimado", caracter_data))

log likelihood:  -44.871674


In [25]:
print("log likelihood: ", log_likelihood(best_model, "Mexico es un pais inseguro", caracter_data))

log likelihood:  -34.247272


La log-verosimilitud es una medida de cuán probable es que una secuencia de texto dada sea generada por un modelo de lenguaje. Valores más altos de log-verosimilitud indican que el modelo asigna una mayor probabilidad a la secuencia de texto observada, lo que sugiere que el modelo piensa que es más "verosímil" o probable.

Cuando se calcula la log-verosimilitud, se suman los logaritmos de las probabilidades asignadas a las palabras reales. Dado que estas probabilidades están entre 0 y 1, sus logaritmos son negativos. Por lo tanto, una log-verosimilitud más alta (menos negativa) significa que las probabilidades asociadas a las palabras observadas son más altas, y el modelo considera que la secuencia es más probable. Una log-verosimilitud muy negativa indica que el modelo asigna muy bajas probabilidades a las palabras observadas, lo que sugiere que el modelo considera que esa secuencia de texto es poco probable.

Debido al corpus con el que se construyó el modelo, las frases mas "agresivas" fueron mas probables ya que tienen un valor de log-verosimilitud mas cercano a cero.

### Estructura morfológica

Evaluamos la permutación de la palabra "habia"

In [26]:
from itertools import permutations
from random import shuffle

word_list = "Habia"
perms = [" ".join(perm) for perm in permutations(word_list)]
#print(len(perms))
print("-"*50)
for p, t in sorted([(log_likelihood(best_model, text, caracter_data), text) for text in perms], reverse = True)[:5]:
    print(p, t)
print("-"*50)  
for p, t in sorted([(log_likelihood(best_model, text, caracter_data), text) for text in perms], reverse = True)[-5:]:
    print(p, t)

--------------------------------------------------
-13.483011 H i a a b
-13.527386 H b i a a
-13.591076 H b a a i
-13.669549 H i a a b
-14.227919 i a H a b
--------------------------------------------------
-29.35515 b a H i a
-29.366308 a a i b H
-29.491879 a a b i H
-29.68933 a a i H b
-30.166454 a a b i H


## Ejercicio 2
**2. Con base en la implementación mostrada en clase, construya un modelo de lenguaje neuronal a nivel de palabra, pero preinicializado con los embeddings proporcionados. Tomé en cuenta secuencias de tamaño 4 para el modelo, es decir hasta 3 palabras en el contexto. Después de haber entrenado el modelo, recupere las 10 palabras más similares a tres palabras de su gusto dadas. Ponga al modelo a generar texto a partir de tres secuencias de inicio de su gusto. Escriba 5 ejemplos de oraciones y mídales el likelihood. Proponga un ejemplo para ver estructuras sintácticas (permutaciones de palabras de alguna oración) buenas usando el likelihood a partir de una oración que usted proponga. Calcule la perplejidad del modelo sobre los datos val. Compárelo con la perplejidad del modelo de lenguaje sin embeddings preentrenados (el visto en clase). DISCUTA BREVEMENTE.**

Para realizar este ejercicio, necesitamos construir y entrenar un modelo de lenguaje neuronal, utilizando embeddings preentrenados que se nos proporciona para este ejercicio.

Necesitamos cargar los embeddings y agregarlos en el modelo.

In [27]:
args = Namespace()
args.N = 4

class NgramData():
    # Inicialización de la clase NgramData
    def __init__(self, N:int, vocab_max: int = 5000, tokenizer = None, embeddings_model = None):
        # Asignación de un tokenizador personalizado o el predeterminado si no se proporciona
        self.tokenizer = tokenizer if tokenizer else self.default_tokenizer
        # Definición de signos de puntuación a ignorar
        self.punct = set(['.', ',', ';', ':', '-', '_', '!', '¡', '?', '¿', '^', '<url>', '*', '@usuario'])
        # Número de palabras en cada n-grama
        self.N = N
        # Tamaño máximo del vocabulario
        self.vocab_max = vocab_max
        # Token para palabras desconocidas
        self.UNK = '<unk>'
        # Token para indicar el inicio de una secuencia
        self.SOS = '<s>'
        # Token para indicar el final de una secuencia
        self.EOS = '</s>'
        # Modelo de embeddings (opcional)
        self.embeddings_model = embeddings_model

    # Función para obtener el tamaño del vocabulario
    def get_vocab_size(self) -> int:
        return len(self.vocab)

    # Tokenizador predeterminado que divide el texto por espacios
    def default_tokenizer(self, doc: str) -> list:
        return doc.split(" ")
    
    # Función para determinar si una palabra debe eliminarse (basada en puntuación o si es un número)
    def remove_word(self, word: str) -> bool:
        word = word.lower()
        is_punct = word in self.punct
        is_digit = word.isnumeric()
        return is_punct or is_digit
    
    # Construye el vocabulario a partir de un corpus, excluyendo palabras según `remove_word` y limitando el tamaño
    def get_vocab(self, corpus: list) -> set:
        # Construcción de la distribución de frecuencia de palabras
        freq_dist = FreqDist([w.lower() for sentence in corpus for w in self.tokenizer(sentence) if not self.remove_word(w)])
        # Ordenar palabras por frecuencia y limitar el tamaño del vocabulario
        sorted_words = self.sortFreqDict(freq_dist)[:self.vocab_max-3]
        return set(sorted_words)
    
    # Ordena el diccionario de frecuencia de palabras
    def sortFreqDict(self, freq_dist) -> list:
        freq_dict = dict(freq_dist)
        return sorted(freq_dict, key=freq_dict.get, reverse=True)
    
    # Ajusta el modelo al corpus, construyendo vocabulario, mapeos de palabras a ID y opcionalmente una matriz de embeddings
    def fit(self, corpus: list) -> None:
        self.vocab = self.get_vocab(corpus)
        # Agregar tokens especiales al vocabulario
        self.vocab.update({self.UNK, self.SOS, self.EOS})
        
        self.w2id = {}
        self.id2w = {}
        
        # Opcional: inicialización de la matriz de embeddings
        if self.embeddings_model is not None:
            self.embedding_matrix = np.empty([len(self.vocab), self.embeddings_model.vector_size])
            
        id = 0
        for doc in corpus:
            for word in self.tokenizer(doc):
                word_ = word.lower()
                if word_ in self.vocab and word_ not in self.w2id:
                    self.w2id[word_] = id
                    self.id2w[id] = word_
                    # Si se proporciona un modelo de embeddings, asignar vector o vector aleatorio si la palabra no está en el modelo
                    if self.embeddings_model is not None:
                        self.embedding_matrix[id] = self.embeddings_model[word_] if word_ in self.embeddings_model else np.random.rand(self.embeddings_model.vector_size)
                    id += 1
        
        # Actualizar mapeos con tokens especiales
        self.w2id.update({self.UNK: id, self.SOS: id+1, self.EOS: id+2})
        self.id2w.update({id: self.UNK, id+1: self.SOS, id+2: self.EOS})

    
    # Transforma el corpus en secuencias de entrada (X_ngrams) y etiquetas objetivo (y) para el modelado
    def transform(self, corpus: list) -> Tuple[np.ndarray, np.ndarray]:
        X_ngrams = []  # Lista para almacenar secuencias de entrada
        y = []  # Lista para almacenar las etiquetas objetivo

        # Iterar sobre cada documento en el corpus
        for doc in corpus:
            # Obtener n-gramas para el documento actual
            doc_ngram = self.get_ngram_doc(doc)
            # Iterar sobre cada ventana de palabras (n-grama) en el documento
            for words_window in doc_ngram:
                # Convertir palabras en IDs usando el mapeo palabra-ID
                words_window_ids = [self.w2id[w] for w in words_window]
                # Las primeras N-1 palabras son la entrada, la última palabra es la etiqueta objetivo
                X_ngrams.append(list(words_window_ids[:-1]))
                y.append(words_window_ids[-1])
        # Convertir las listas en arrays de NumPy para su uso en modelos de aprendizaje automático
        return np.array(X_ngrams), np.array(y)
    
    # Genera n-gramas a partir de un documento, preparándolo para la transformación
    def get_ngram_doc(self, doc: str) -> list:
        # Tokenizar el documento
        doc_tokens = self.tokenizer(doc)
        # Reemplazar tokens desconocidos por el token <unk>
        doc_tokens = self.replace_unk(doc_tokens)
        # Convertir todos los tokens a minúsculas
        doc_tokens = [w.lower() for w in doc_tokens]
        # Añadir tokens de inicio (<s>) y fin (</s>) al documento
        doc_tokens = [self.SOS]*(self.N-1) + doc_tokens + [self.EOS]
        # Generar y retornar n-gramas del documento procesado
        return list(ngrams(doc_tokens, self.N))
    
    # Reemplaza tokens desconocidos en una lista de tokens por el token <unk>
    def replace_unk(self, doc_tokens: list) -> list:
        # Iterar sobre cada token en la lista de tokens
        for i, token in enumerate(doc_tokens):
            # Si el token no está en el vocabulario, reemplazarlo por <unk>
            if token.lower() not in self.vocab:
                doc_tokens[i] = self.UNK
        # Retornar la lista de tokens con los desconocidos reemplazados
        return doc_tokens


Necesitamos ahora cargar los embeddings. Implementamos una función para esto.

In [28]:
def load_embeddings(file_path):
    embeddings_dict = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], "float32")
            embeddings_dict[word] = vector
    return embeddings_dict

embeddings_dict = load_embeddings("word2vec_col.txt")

Ahora necesitamos construir el modelo, integramos el embedding en la clase que construye el modelo como objeto.

In [29]:
tk = TweetTokenizer() # Inicializamos el tokenizador de tweets

# Creamos un objeto con la clase que definimos
ngram_data = NgramData(args.N, 5000, tk.tokenize)
ngram_data.fit(X_train) # Construye el vocabulario

# Transformación de datos
X_ngram_train, y_ngram_train = ngram_data.transform(X_train)
X_ngram_val, y_ngram_val = ngram_data.transform(X_val)

In [30]:
# Pytorch 

args.batch_size = 64
# Num workers
args.num_workers = 2

# Convertimos los datos a tensores de pytorch
train_dataset = TensorDataset(torch.tensor(X_ngram_train, dtype = torch.int64),
                              torch.tensor(y_ngram_train, dtype = torch.int64))

train_loader = DataLoader(train_dataset,
                          batch_size = args.batch_size,
                          num_workers=args.num_workers,
                          shuffle = True)

val_dataset = TensorDataset(torch.tensor(X_ngram_val, dtype = torch.int64),
                            torch.tensor(y_ngram_val, dtype = torch.int64))

val_loader = DataLoader(train_dataset,
                          batch_size = args.batch_size,
                          num_workers=args.num_workers,
                          shuffle = False)

batch = next(iter(train_loader))
print(f'X shape: {batch[0].shape}')
print(f'y shape: {batch[1].shape}')

X shape: torch.Size([64, 3])
y shape: torch.Size([64])


In [31]:
# Vocab size
args.vocab_size = ngram_data.get_vocab_size()

# Dimension of word embeddings
args.d = 100

# Dimension for hidden layer
args.d_h = 100

# Dropout
args.dropout = 0.1

Preparamos una matriz de embeddings que se pueda usar para inicializar la capa de embeddings en el modelo de PyTorch. 

In [32]:
embedding_dim = args.d
vocab_size = args.vocab_size

# Obteniendo el mapeo de palabra a ID desde tu instancia de NgramData
word_to_id = ngram_data.w2id

# Inicializa la matriz de embeddings con ceros
embeddings_matrix = np.zeros((vocab_size, embedding_dim))

# Llena la matriz con los embeddings para cada palabra en el vocabulario
for word, i in word_to_id.items():
    embedding_vector = embeddings_dict.get(word)
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector


In [33]:
class NeuralLM(nn.Module):
    def __init__(self, args, embeddings_matrix):
        super(NeuralLM, self).__init__()
        
        # Tamaño de la ventana de entrada (contexto) para el modelo, basado en N-1
        # donde N es el tamaño de los n-gramas considerados.
        self.window_size = args.N-1
        
        # Dimensión de los embeddings de palabras, que transforman índices de palabras
        # en vectores densos que capturan información semántica.
        self.embedding_dim = args.d
        
        # Inicializa capa de embeddings
        self.emb = nn.Embedding(args.vocab_size, self.embedding_dim)

        # Convierte embeddings_matrix de NumPy array a PyTorch tensor
        embeddings_tensor = torch.tensor(embeddings_matrix, dtype=torch.float)

        self.emb.weight.data.copy_(embeddings_tensor)  # Establece los pesos de la capa de embedding
        
        # Otras capas
        self.fc1 = nn.Linear(self.embedding_dim * self.window_size, args.d_h)
        self.drop1 = nn.Dropout(p=args.dropout)
        self.fc2 = nn.Linear(args.d_h, args.vocab_size, bias=False)
        
    def forward(self, x):
        # Transforma los índices de palabras en x en embeddings de palabras.
        x = self.emb(x)
        
        # Aplana los embeddings en un vector único para cada muestra en el lote,
        # preparándolos para la entrada a la capa lineal.
        x = x.view(-1, self.window_size * self.embedding_dim)
        
        # Pasa la entrada aplanada a través de la primera capa lineal y luego
        # a través de una función de activación ReLU.
        h = F.relu(self.fc1(x))
        
        # Aplica Dropout a la representación de la capa oculta para mejorar la generalización.
        h = self.drop1(h)
        
        # La salida final se obtiene pasando la representación de la capa oculta después de Dropout
        # a través de la segunda capa lineal, generando los logits para cada palabra en el vocabulario.
        return self.fc2(h)


In [34]:
# Esta función toma los logits (es decir, las salidas no normalizadas de un modelo) 
# y devuelve las predicciones de clase como índices.
def get_preds(raw_logits):
    # Calcula las probabilidades aplicando la función softmax a los logits.
    # La operación detach() se usa para evitar que se calculen gradientes para estas operaciones,
    # ya que solo se necesitan las probabilidades para hacer predicciones.
    probs = F.softmax(raw_logits.detach(), dim=1)
    
    # Encuentra el índice de la mayor probabilidad en cada fila (es decir, para cada ejemplo en el lote),
    # que corresponde a la clase predicha. Luego, convierte el tensor a un array de NumPy.
    y_pred = torch.argmax(probs, dim=1).cpu().numpy()
    
    return y_pred


# Evalúa el modelo en un conjunto de datos proporcionado y devuelve la precisión del modelo.
def model_eval(data, model, gpu=False):
    # Desactiva el cálculo de gradientes para acelerar las cosas y reducir el uso de memoria
    # ya que no se necesita para la evaluación.
    with torch.no_grad():
        preds, tgts = [], []  # Listas para almacenar predicciones y etiquetas verdaderas
        
        # Itera sobre los lotes de datos en el DataLoader
        for window_words, labels in data:
            # Si se utiliza GPU, mueve los datos al dispositivo adecuado
            if gpu:
                window_words = window_words.cuda()
                
            # Obtiene los logits del modelo para el lote actual
            outputs = model(window_words)
            
            # Obtiene las predicciones de clase para el lote actual utilizando la función get_preds
            y_pred = get_preds(outputs)
            
            # Extrae las etiquetas verdaderas del lote actual y las convierte a un array de NumPy
            tgt = labels.numpy()
            
            # Almacena las predicciones y las etiquetas verdaderas
            tgts.append(tgt)
            preds.append(y_pred)
        
        # Aplana las listas de listas para obtener una única lista de etiquetas y predicciones
        tgts = [e for l in tgts for e in l]
        preds = [e for l in preds for e in l]
        
        # Calcula y devuelve la precisión del modelo comparando las predicciones con las etiquetas verdaderas
        return accuracy_score(tgts, preds)
    

# Guarda el estado actual del modelo y, si es el mejor modelo hasta el momento según 
# algún criterio, guarda una copia separada.
def save_checkpoint(state, is_best, checkpoint_path, filename="checkpoint.pt"):
    # Construye la ruta completa del archivo donde se guardará el estado del modelo
    filename = os.path.join(checkpoint_path, filename)
    
    # Guarda el estado del modelo en la ruta especificada
    torch.save(state, filename)
    
    # Si el modelo actual es el "mejor" según algún criterio, guarda una copia separada
    if is_best:
        # Copia el archivo del checkpoint al archivo del "mejor modelo"
        shutil.copyfile(filename, os.path.join(checkpoint_path, "model_best.pt"))



Definición de modelo e hiperparámetros.

In [35]:
# Model hyperparameters
args.vocab_size = ngram_data.get_vocab_size() 
args.d = 100
args.d_h = 200
args.dropout = 0.1

# Training hyperparameters
args.lr = 2.3e-1
args.num_epochs = 20
args.patience = 20

# Scheduler hyperparameters
args.lr_patience = 10
args.lr_factor = 0.5

# Saving directoty
args.savedir = 'model'
os.makedirs(args.savedir, exist_ok=True)

# Create model
model = NeuralLM(args, embeddings_matrix)

# Send to GPU
args.use_gpu = torch.cuda.is_available()
if args.use_gpu:
    model.cuda()
    
# Loss, optimizer an scheduler
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr= args.lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, 'min',
                patience=args.lr_patience,
                verbose=True,
                factor=args.lr_factor
                )

/Users/guillermo_sego/anaconda3/envs/PLN/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Entrenamiento de la red neuronal.

In [36]:
start_time = time.time()
best_metric = 0
metric_history = []
train_metric_history = []

for epoch in range(args.num_epochs):
    epoch_stats_time = time.time()
    loss_epoch = []
    training_metric = []
    model.train()
    
    for window_words, labels in train_loader:
        if args.use_gpu:
            window_words = window_words.cuda()
            labels = labels.cuda()
            
        #Forward pass
        outputs = model(window_words)
        loss = criterion(outputs, labels)
        loss_epoch.append(loss.item())
        
        #Get training metrics
        y_pred = get_preds(outputs)
        tgt = labels.cpu().numpy()
        training_metric.append(accuracy_score(tgt, y_pred))
        
        #Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    #Get metric in training dataset
    mean_epoch_metric = np.mean(training_metric)
    train_metric_history.append(mean_epoch_metric)
    
    #Get metric in validation dataset
    model.eval()
    tuning_metric = model_eval(val_loader, model, gpu=args.use_gpu)
    metric_history.append(mean_epoch_metric)
    
    #update scheduler
    scheduler.step(tuning_metric)
    
    #Check for metric improvement
    is_improvement = tuning_metric > best_metric
    if is_improvement:
        best_metric = tuning_metric
        n_no_improve = 0
    else:
        n_no_improve = 1
        
    save_checkpoint(
        {
            "epoch": epoch + 1,
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict(),
            "scheduler": scheduler.state_dict(),
            "best_metric": best_metric,
        },
        is_improvement,
        args.savedir,
    )
    
    #Early stopping
    if n_no_improve >= args.patience:
        print("No improvement. Breaking out of loop.")
        
    print("Train acc: {}".format(mean_epoch_metric))
    print("Epoch [{}/{}], Loss: {:.4f} - Val accuracy: {:.4f} - Epoch time: {:.2f}"
         .format(epoch+1, args.num_epochs, np.mean(loss_epoch), tuning_metric, (time.time())))
    

print("--- %s seconds ---" % (time.time() - start_time))

Train acc: 0.1483213779978307
Epoch [1/20], Loss: 5.7387 - Val accuracy: 0.1709 - Epoch time: 1711171210.20
Train acc: 0.16737229090507375
Epoch [2/20], Loss: 5.2152 - Val accuracy: 0.1934 - Epoch time: 1711171223.80
Train acc: 0.1714123950508175
Epoch [3/20], Loss: 5.0171 - Val accuracy: 0.1873 - Epoch time: 1711171241.22
Train acc: 0.17458158418029165
Epoch [4/20], Loss: 4.8582 - Val accuracy: 0.1812 - Epoch time: 1711171254.90
Train acc: 0.17931904802554935
Epoch [5/20], Loss: 4.7290 - Val accuracy: 0.1404 - Epoch time: 1711171268.79
Train acc: 0.1808769659141124
Epoch [6/20], Loss: 4.6089 - Val accuracy: 0.1797 - Epoch time: 1711171282.47
Train acc: 0.18392030390069494
Epoch [7/20], Loss: 4.5016 - Val accuracy: 0.2164 - Epoch time: 1711171296.15
Train acc: 0.18688110111276263
Epoch [8/20], Loss: 4.4083 - Val accuracy: 0.1850 - Epoch time: 1711171309.87
Train acc: 0.19044981269834896
Epoch [9/20], Loss: 4.3219 - Val accuracy: 0.2199 - Epoch time: 1711171323.55
Train acc: 0.194615140

Truncamos a 20 debido al tiempo de entrega de la tarea :(

### Evaluación del modelo
Ahora evaluamos y comparamos el modelo con el desarrollado en clase. Primero necesitamos revisar las palabras mas similares en el contexto dado. 

In [37]:
def print_closest_words(embeddings, ngram_data, word, n):
    word_id = torch.LongTensor([ngram_data.w2id[word]])
    word_embed = embeddings(word_id)
    dists = torch.norm(embeddings.weight - word_embed, dim = 1).detach()
    lst = sorted(enumerate(dists.numpy()), key = lambda x: x[1])
    for idx, difference in lst[1:n+1]:
        print(ngram_data.id2w[idx], difference)

Cargamos el mejor modelo que se haya guardado y proseguimos para hacer una visualización

In [38]:
# Model with learned embeddings from scratch
best_model = NeuralLM(args, embeddings_matrix)
best_model.load_state_dict(torch.load("model/model_best.pt")["state_dict"])

print("-"*30)
print("Learned embeddings")
print("-"*30)
print_closest_words(best_model.emb, ngram_data, "bonito", 10)

------------------------------
Learned embeddings
------------------------------
lindo 10.560313
hermoso 15.729565
feo 19.227892
reconfortante 20.34947
gracioso 21.105967
perfecto 21.274488
chistoso 21.696632
chingon 21.848997
chingón 22.155758
guapo 22.157343


Ahora, generamos texto con el modelo

In [39]:
def parse_text(text, tokenizer):
    # Tokeniza el texto y convierte cada palabra a minúsculas. Si la palabra está en el vocabulario (w2id), la usa;
    # de lo contrario, la reemplaza por el token "<unk>" para palabras desconocidas.
    all_tokens = [w.lower() if w.lower() in ngram_data.w2id else "<unk>" for w in tokenizer.tokenize(text)]
    
    # Convierte los tokens a sus índices numéricos correspondientes según el mapeo w2id en ngram_data.
    token_ids = [ngram_data.w2id[word.lower()] for word in all_tokens]
    
    return all_tokens, token_ids


def sample_next_word(logits, temperature=1.0):
    # Convierte los logits a un array de numpy y ajusta la "temperatura" de la predicción.
    logits = np.asarray(logits).astype("float64")
    preds = logits / temperature
    
    # Convierte los logits ajustados a probabilidades usando softmax.
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    
    # Muestrea un índice de palabra de la distribución de probabilidades.
    probas = np.random.multinomial(1, preds)
    return np.argmax(probas)


def predict_next_token(model, token_ids):
    # Convierte la lista de índices de tokens a un tensor de PyTorch y agrega una dimensión de lote.
    word_ids_tensor = torch.LongTensor(token_ids).unsqueeze(0)
    
    # Obtiene los logits de la predicción del modelo para la secuencia de tokens y los convierte a numpy.
    y_raw_pred = model(word_ids_tensor).squeeze(0).detach().numpy()
    
    # Muestra el índice de la siguiente palabra de la distribución de logits.
    y_pred = sample_next_word(y_raw_pred, 1.0)
    return y_pred


def generate_sentence(model, initial_text, tokenizer):
    # Obtiene tokens y sus índices del texto inicial.
    all_tokens, window_word_ids = parse_text(initial_text, tokenizer)
    
    # Genera hasta 100 palabras adicionales.
    for i in range(100):
        # Predice el índice de la siguiente palabra utilizando el modelo.
        y_pred = predict_next_token(model, window_word_ids)
        next_word = ngram_data.id2w[y_pred]  # Convierte el índice de palabra predicho a texto.
        all_tokens.append(next_word)  # Añade la palabra predicha a la lista de tokens.
        
        # Si se genera el token de fin de secuencia, detiene la generación.
        if next_word == "</s>":
            break
        else:
            # Actualiza la ventana de palabras para la siguiente predicción.
            window_word_ids.pop(0)  # Elimina la primera palabra.
            window_word_ids.append(y_pred)  # Añade la nueva palabra al final.
    
    # Une los tokens generados en una cadena de texto y la devuelve.
    return " ".join(all_tokens)


In [40]:
initial_tokens = "<s><s><s>"

print("-"*30)
print("Learned embeddings")
print("-"*30)
print(generate_sentence(best_model, initial_tokens, tk))

------------------------------
Learned embeddings
------------------------------
<s> <s> <s> vea al puto pendejo <unk> mi sangre <unk> <unk> … </s>


Ahora, medimos el valor del likelihood del modelo.

In [41]:
def log_likelihood(model, text, ngram_model):
    # Transforma el texto dado en n-gramas (X) y las etiquetas objetivo (y) utilizando el modelo n-gram.
    X, y = ngram_data.transform([text])
    
    # Ignora los primeros dos n-gramas. Esto podría ser específico para cómo se estructura el texto o los n-gramas.
    X, y = X[2:], y[2:]
    
    # Convierte X en un tensor de PyTorch y agrega una dimensión de lote, preparándolo para el modelo.
    X = torch.LongTensor(X).unsqueeze(0)
    
    # Obtiene los logits del modelo para el texto transformado y luego los desconecta del grafo de cómputo.
    logits = model(X).detach()
    
    # Aplica softmax a los logits para obtener una distribución de probabilidades sobre el vocabulario.
    probs = F.softmax(logits, dim=1).numpy()
    
    # Calcula la log-verosimilitud sumando los logaritmos de las probabilidades de las palabras reales (y)
    # según lo predicho por el modelo.
    return np.sum([np.log(probs[i][w]) for i, w in enumerate(y)])


In [42]:
print("log likelihood: ", log_likelihood(best_model, "Estamos en la clase de procesamiento de lenguaje", ngram_data))

log likelihood:  -28.747715


In [43]:
print("log likelihood: ", log_likelihood(best_model, "Sergio Perez es el mejor piloto de f1", ngram_data))

log likelihood:  -24.890015


In [44]:
print("log likelihood: ", log_likelihood(best_model, "Mexico esta en decadencia por los narcos", ngram_data))

log likelihood:  -42.84663


In [45]:
print("log likelihood: ", log_likelihood(best_model, "Tonto, viejo malo, que te pasa", ngram_data))

log likelihood:  -40.24046


In [46]:
print("log likelihood: ", log_likelihood(best_model, "Horrible, todo esta muy mal, que horrible", ngram_data))

log likelihood:  -54.03869


### Estructura morfológica

Evaluamos las permutaciones del modelo

In [47]:
from itertools import permutations
from random import shuffle

word_list = "sino gano me voy a la chingada".split(" ")
perms = [" ".join(perm) for perm in permutations(word_list)]
#print(len(perms))
print("-"*50)
for p, t in sorted([(log_likelihood(best_model, text, ngram_data), text) for text in perms], reverse = True)[:5]:
    print(p, t)
print("-"*50)  
for p, t in sorted([(log_likelihood(best_model, text, ngram_data), text) for text in perms], reverse = True)[-5:]:
    print(p, t)

--------------------------------------------------


-22.08913 sino gano me voy a la chingada
-24.907225 voy sino me gano a la chingada
-25.02383 sino voy me gano a la chingada
-25.140871 gano sino me voy a la chingada
-25.212936 gano sino me voy la chingada a
--------------------------------------------------
-82.64427 a la voy gano chingada sino me
-83.02557 a la gano voy sino chingada me
-84.76806 a la gano voy chingada me sino
-86.985146 a la voy gano chingada me sino
-89.53535 la voy gano chingada a me sino


Lo comparamos con el modelo de la clase. Los valores de la perplejidad fueron mejores en el modelo con embeddings. La integración de embeddings preentrenados en el modelo de lenguaje neuronal muestra una mejora significativa en la coherencia y naturalidad de las secuencias de palabras generadas, como evidencian las perplejidades reducidas tanto en oraciones bien estructuradas como en aquellas con estructuras sintácticas menos convencionales. La disminución de la perplejidad con embeddings preentrenados sugiere una comprensión semántica más rica y una capacidad mejorada para capturar contextos y relaciones entre palabras, lo que resulta en un modelo más robusto y eficaz en la generación de texto natural. Este contraste resalta el valor de los embeddings preentrenados en enriquecer modelos de lenguaje con conocimiento semántico previo, facilitando así una generación de lenguaje más coherente y fluida.

# Ejercicio 3

**3. (OPCIONAL: 30pts extra en ESTA tarea, calificación máxima para promediar con otras tareas: 130) A partir del modelo anterior haga un modelo de lenguaje que integre una conexión directa de la capa de embeddings hacía la salida, justo como lo proponía Bengio. Discuta sobre las diferencias en el proceso de entrenamiento y la perplejidad respecto al modelo anterior y el visto en clase.**

Para poder agregar una capa extra al embedding hay que modidicar la clase **NeuralLM** para integrar la capa extra propuesta por el modelo de Bengio. En este modelo modificado, hay dos caminos desde la capa de embeddings hacia la salida:

* Conexión Directa: Una conexión lineal directa desde los embeddings aplanados hasta la salida. Esto está representado por la capa self.direct_conn. Esta conexión intenta capturar las relaciones lineales entre las palabras y su contexto.

* Ruta No Lineal: La ruta original que pasa a través de una capa oculta (y potencialmente más capas) antes de llegar a la salida. Esto permite que el modelo capture relaciones no lineales complejas.

La salida final del modelo es la suma de las salidas de ambos caminos.

In [48]:
class NeuralLM(nn.Module):
    def __init__(self, args, embeddings_matrix):
        super(NeuralLM, self).__init__()
        
        self.window_size = args.N - 1
        self.embedding_dim = args.d
        
        # Capa de embedding
        self.emb = nn.Embedding(args.vocab_size, self.embedding_dim)
        
        # Convierte la matriz de NumPy a un tensor de PyTorch y asigna como pesos preentrenados
        embeddings_tensor = torch.tensor(embeddings_matrix, dtype=torch.float32)
        self.emb.weight = nn.Parameter(embeddings_tensor, requires_grad=False)  
        
        # Capa lineal para la conexión directa desde los embeddings a la salida
        self.direct_conn = nn.Linear(self.embedding_dim * self.window_size, args.vocab_size)
        
        # Capas para la ruta no lineal
        self.fc1 = nn.Linear(self.embedding_dim * self.window_size, args.d_h)
        self.drop1 = nn.Dropout(p=args.dropout)
        self.fc2 = nn.Linear(args.d_h, args.vocab_size)

    def forward(self, x):
        # Transforma los índices de palabras en x en embeddings de palabras
        embeddings = self.emb(x)
        
        # Aplana los embeddings para pasarlos a través de las capas lineales
        flattened = embeddings.view(-1, self.window_size * self.embedding_dim)
        
        # Conexión directa de embeddings a salida
        direct_output = self.direct_conn(flattened)
        
        # Ruta no lineal
        h = F.relu(self.fc1(flattened))
        h = self.drop1(h)
        non_linear_output = self.fc2(h)
        
        # Suma las salidas de las conexiones directa y no lineal
        combined_output = direct_output + non_linear_output
        
        return combined_output


Definición e hiperparámetros

In [49]:
# Model hyperparameters
args.vocab_size = ngram_data.get_vocab_size() 
args.d = 100
args.d_h = 200
args.dropout = 0.1

# Training hyperparameters
args.lr = 2.3e-1
args.num_epochs = 20
args.patience = 20

# Scheduler hyperparameters
args.lr_patience = 10
args.lr_factor = 0.5

# Saving directoty
args.savedir = 'model'
os.makedirs(args.savedir, exist_ok=True)

# Create model
model = NeuralLM(args, embeddings_matrix)

# Send to GPU
args.use_gpu = torch.cuda.is_available()
if args.use_gpu:
    model.cuda()
    
# Loss, optimizer an scheduler
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr= args.lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, 'min',
                patience=args.lr_patience,
                verbose=True,
                factor=args.lr_factor
                )

/Users/guillermo_sego/anaconda3/envs/PLN/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Entrenamiento

In [50]:
start_time = time.time()
best_metric = 0
metric_history = []
train_metric_history = []

for epoch in range(args.num_epochs):
    epoch_stats_time = time.time()
    loss_epoch = []
    training_metric = []
    model.train()
    
    for window_words, labels in train_loader:
        if args.use_gpu:
            window_words = window_words.cuda()
            labels = labels.cuda()
            
        #Forward pass
        outputs = model(window_words)
        loss = criterion(outputs, labels)
        loss_epoch.append(loss.item())
        
        #Get training metrics
        y_pred = get_preds(outputs)
        tgt = labels.cpu().numpy()
        training_metric.append(accuracy_score(tgt, y_pred))
        
        #Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    #Get metric in training dataset
    mean_epoch_metric = np.mean(training_metric)
    train_metric_history.append(mean_epoch_metric)
    
    #Get metric in validation dataset
    model.eval()
    tuning_metric = model_eval(val_loader, model, gpu=args.use_gpu)
    metric_history.append(mean_epoch_metric)
    
    #update scheduler
    scheduler.step(tuning_metric)
    
    #Check for metric improvement
    is_improvement = tuning_metric > best_metric
    if is_improvement:
        best_metric = tuning_metric
        n_no_improve = 0
    else:
        n_no_improve = 1
        
    save_checkpoint(
        {
            "epoch": epoch + 1,
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict(),
            "scheduler": scheduler.state_dict(),
            "best_metric": best_metric,
        },
        is_improvement,
        args.savedir,
    )
    
    #Early stopping
    if n_no_improve >= args.patience:
        print("No improvement. Breaking out of loop.")
        
    print("Train acc: {}".format(mean_epoch_metric))
    print("Epoch [{}/{}], Loss: {:.4f} - Val accuracy: {:.4f} - Epoch time: {:.2f}"
         .format(epoch+1, args.num_epochs, np.mean(loss_epoch), tuning_metric, (time.time())))
    

print("--- %s seconds ---" % (time.time() - start_time))

Train acc: 0.11172097828706866
Epoch [1/20], Loss: 8.8886 - Val accuracy: 0.1493 - Epoch time: 1711171504.28
Train acc: 0.12053024444622985
Epoch [2/20], Loss: 7.8851 - Val accuracy: 0.1377 - Epoch time: 1711171524.59
Train acc: 0.127338132205841
Epoch [3/20], Loss: 7.6160 - Val accuracy: 0.1521 - Epoch time: 1711171547.32
Train acc: 0.13876861718153696
Epoch [4/20], Loss: 7.3455 - Val accuracy: 0.1675 - Epoch time: 1711171566.42
Train acc: 0.15113623809906399
Epoch [5/20], Loss: 7.1834 - Val accuracy: 0.1622 - Epoch time: 1711171589.28
Train acc: 0.16253628027959668
Epoch [6/20], Loss: 7.0548 - Val accuracy: 0.1890 - Epoch time: 1711171613.84
Train acc: 0.1731149946772185
Epoch [7/20], Loss: 6.9522 - Val accuracy: 0.1959 - Epoch time: 1711171637.29
Train acc: 0.18045076678182623
Epoch [8/20], Loss: 6.8860 - Val accuracy: 0.1979 - Epoch time: 1711171657.94
Train acc: 0.18811670198449362
Epoch [9/20], Loss: 6.8445 - Val accuracy: 0.1805 - Epoch time: 1711171679.59
Train acc: 0.194890066

### Evaluación del modelo
Ahora comparamos el modelo con el realizado en el inciso anterior.

In [51]:
# Model with learned embeddings from scratch
best_model = NeuralLM(args, embeddings_matrix)
best_model.load_state_dict(torch.load("model/model_best.pt")["state_dict"])

print("-"*30)
print("Learned embeddings")
print("-"*30)
print_closest_words(best_model.emb, ngram_data, "bonito", 10)

------------------------------
Learned embeddings
------------------------------
lindo 10.613454
hermoso 15.747205
feo 19.22297
reconfortante 20.328833
gracioso 21.1011
perfecto 21.245314
chistoso 21.689884
chingon 21.850063
chingón 22.149546
genial 22.179726


In [52]:
initial_tokens = "<s><s><s>"

print("-"*30)
print("Learned embeddings")
print("-"*30)
print(generate_sentence(best_model, initial_tokens, tk))

------------------------------
Learned embeddings
------------------------------
<s> <s> <s> me queda claro <unk> </s>


In [53]:
print("log likelihood: ", log_likelihood(best_model, "Estamos en la clase de procesamiento de lenguaje", ngram_data))

log likelihood:  -34.184837


In [54]:
print("log likelihood: ", log_likelihood(best_model, "Sergio Perez es el mejor piloto de f1", ngram_data))

log likelihood:  -49.329678


In [55]:
print("log likelihood: ", log_likelihood(best_model, "Mexico esta en decadencia por los narcos", ngram_data))

log likelihood:  -44.581127


In [56]:
print("log likelihood: ", log_likelihood(best_model, "Tonto, viejo malo, que te pasa", ngram_data))

log likelihood:  -53.303963


In [57]:
print("log likelihood: ", log_likelihood(best_model, "Horrible, todo esta muy mal, que horrible", ngram_data))

log likelihood:  -44.472637


### Comparación estructura morfológica

In [58]:
word_list = "sino gano me voy a la chingada".split(" ")
perms = [" ".join(perm) for perm in permutations(word_list)]
#print(len(perms))
print("-"*50)
for p, t in sorted([(log_likelihood(best_model, text, ngram_data), text) for text in perms], reverse = True)[:5]:
    print(p, t)
print("-"*50)  
for p, t in sorted([(log_likelihood(best_model, text, ngram_data), text) for text in perms], reverse = True)[-5:]:
    print(p, t)

--------------------------------------------------
-38.66899 sino gano a la chingada me voy
-38.979294 chingada sino gano a la me voy
-40.775017 gano sino la chingada me voy a
-42.651367 gano chingada a la me voy sino
-42.77483 chingada sino gano la me voy a
--------------------------------------------------
-133.49619 me la a sino chingada voy gano
-136.30128 la me a chingada sino voy gano
-136.89542 me a chingada sino gano voy la
-141.6242 me la a sino voy gano chingada
-142.57162 me la a sino gano voy chingada


### Discusión sobre el entrenamiento y la perplejidad:

La comparación entre los modelos con y sin la conexión directa desde la capa de embeddings hacia la salida revela diferencias significativas en los valores de perplejidad, reflejando el impacto de la arquitectura del modelo en su capacidad para entender y generar secuencias de palabras coherentes.

El modelo **con embeddings preentrenados**, pero **sin la conexión directa**, muestra perplejidades más bajas tanto para oraciones coherentes como para aquellas con estructuras menos convencionales. Esto indica una mejora en la capacidad del modelo para generar secuencias de palabras naturales y coherentes, probablemente debido a la rica información semántica proporcionada por los embeddings.

Por otro lado, el modelo **con la conexión directa** muestra un aumento en los valores de perplejidad para ambos tipos de oraciones. Aunque esta arquitectura, inspirada en Bengio, tiene el potencial de capturar tanto relaciones lineales como no lineales directamente desde los embeddings, el aumento en la perplejidad sugiere que podría estar enfrentando desafíos, como el sobreajuste o la dificultad para integrar efectivamente esta información adicional en la generación de secuencias de palabras.

- La **disminución de la perplejidad** en el modelo con embeddings, pero sin la conexión directa, subraya cómo una comprensión semántica más profunda facilitada por los embeddings puede mejorar la coherencia del texto generado.
- El **aumento de la perplejidad** en el modelo con la conexión directa sugiere que la integración de esta ruta adicional en el modelo no está siendo tan efectiva como se esperaba, lo que podría deberse a varios factores, como la complejidad adicional en la arquitectura o la necesidad de una regulación más fina para evitar el sobreajuste.
- Estas observaciones subrayan la importancia de equilibrar la riqueza semántica proporcionada por los embeddings preentrenados con la complejidad arquitectónica del modelo, asegurando que las adiciones a la arquitectura contribuyan efectivamente a la tarea de generación de lenguaje sin introducir dificultades adicionales en el aprendizaje.